# Aquecimento para a formação em infraestrutura

## Etapas

Aqui serão descritas as etapas do desafio que, idealmente, devem ser seguidas de forma sequencial. Para o caso de dificuldade em alguma etapa, você poderá seguir para a próxima, se assim desejar.

### Etapa 1: Entendendo os dados 🎲

- **Objetivo:** nessa etapa, você deve somente ingerir dados da API do `randomuser.me` e observar o formato dos dados, tentando imaginar como eles poderiam ser usados para construir uma tabela.
- **Descrição da solução:** a solução dessa etapa consiste em uma função para consumir a API na URL `https://randomuser.me/api/` e retornar um dicionário com os dados.
- **Links úteis:**
  - Documentação da API: https://randomuser.me/documentation
  - Introdução a ingestão de dados via API: https://www.dataquest.io/blog/python-api-tutorial/


In [5]:
## Importando a biblioteca
import requests

## Fazendo a requisição à API e recebendo o objeto de retorno na variavel "r"....esse é um objeto da biblioteca request
r = requests.get( 'https://randomuser.me/api/' )

## tranformando o objeto request em string e imprimindo na tela.
print( r.text )


<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>randomuser.me | 520: Web server is returning an unknown error</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">

    

    <div id="cf-error-details" class="p-0">
        <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">
            <h1 class="inline-block sm:block sm:mb-2 font-light text-60 lg:text-4xl 

### Etapa 2: Coletando dados 💾

- **Objetivo:** nessa etapa, você deve coletar dados da API e armazená-los em um arquivo CSV.
- **Descrição da solução:** a solução dessa etapa consiste em uma função para coletar uma quantidade `n` de dados da API (sendo `n` um valor fornecido via parâmetro da função), manipulá-los para montar um `pandas.DataFrame` e salvar o resultado em um arquivo CSV.
- **Links úteis:**
  - Documentação da API: https://randomuser.me/documentation
  - Documentação do Pandas: https://pandas.pydata.org/docs/
- **Dicas:**
  - Para tornar os dados mais fáceis de manipular no futuro, faça com que o `DataFrame` seja "plano", ou seja, cada coluna seja um único atributo do objeto.
  - Para ter dados suficientes para uma análise razoável nas próximas etapas, recomendamos `n>=500`.

In [6]:
## Importando as bibliotecas
import requests
import pandas as pd
import json
from pandas import json_normalize

## Fazendo a requisição à API e setando um n de 600 registros
## r será um objeto da biblioteca request
r = requests.get( 'https://randomuser.me/api/?results=1500' )


In [7]:
## Tranformar o objeto r em uma string json 
dict= json.loads(r.text)

## Normalizar a string json, pois existem objetos aninhados. Estes receberam o separador "_" 
df = json_normalize( dict['results'], sep = "_" )
df.to_csv( "registros_api.csv" , sep=';',encoding='utf8', index=False)
# print(df)

### Etapa 3: Manipulando dados 📝

- **Objetivo**: agora, você pode observar que, na base de dados obtida, devido às diferentes nacionalidades dos usuários, os números de telefone e celular têm formatos diferentes. Você deve transformá-los para um formato único, escolhido arbitrariamente.
- **Descrição da solução**: uma função que recebe, como parâmetro, um `pandas.DataFrame` e retorna um `pandas.DataFrame` com as mesmas colunas, mas com os números de telefone e celular formatados de forma única.
- **Links úteis:**
- Documentação do Pandas: https://pandas.pydata.org/docs/


In [8]:
def format_phone_field(df, colname):

    if colname in df.columns:
        
        def formatPhone(x):
            x = ''.join(c for c in x if c.isdigit())   # Removendo os caracters não numéricos
            x = x.zfill(8)   # Completando com zeros a esquerda caso tenha menos de 7 numeros
            x = x[-8:]   # pegando os 7 números da direita para a esquerda
            x = x[:4] + '-' + x[4:]
            return x   # retorno da função

        df[colname] = df[colname].apply(formatPhone) # Aplicando a função na coluna do dataframe
        
    else:
        print("Variavel nao existe!")
    
## Aplicando a função de formatação dos números de telefone e celular nas respectivas colunas do dataframe
format_phone_field(df, "phone")
format_phone_field(df, "cell")

### Etapa 4: Analisando dados sem agrupamento 📊

- **Objetivo**: com seus dados devidamente tratados, você deve gerar os seguintes itens:
  - Um relatório em texto (não precisa de formatação) contendo:
    - A porcentagem dos usuários por gênero
    - A porcentagem dos usuários por país
  - Uma imagem contendo um gráfico de distribuição da idade dos usuários (a biblioteca utilizada para o `plot` pode ser qualquer uma).
- **Descrição da solução**: uma função que recebe, como parâmetro, um `pandas.DataFrame` e gera dois arquivos: um relatório em texto e outro contendo um gráfico de distribuição da idade dos usuários.
- **Links úteis:**
  - Documentação do Pandas: https://pandas.pydata.org/docs/
  - Documentação do Matplotlib: https://matplotlib.org/
  - Documentação do Seaborn: https://seaborn.pydata.org/

In [9]:
import numpy as np
import seaborn as sns
sns.set_theme(style="ticks")

df_gender = df.gender.value_counts().rename_axis('gender').to_frame('total').reset_index()
df_gender["percentage"] = round ( df_gender.total / df_gender.total.sum() * 100, 0 )
df_gender

df_country = df.location_country.value_counts().rename_axis('country').to_frame('total').reset_index()
df_country["percentage"] = round ( df_country.total / df_country.total.sum() * 100, 0 )
df_country

html = df_country.to_html()

with open("aquecimento.html", 'w') as arquivo:
    arquivo.write( 
        "<h1> Relatório Etapa 04</h1> \n" +
        "<h3> Percentual por sexo</h3>" +
        df_gender.to_html() + "\n\n\n" +
        "<h3> Percentual por páis</h3>" +
        df_country.to_html()
    )

## Escolhendo os binds
print( df['dob_age'].min() ) ## 21
print(  df['dob_age'].max()) ## 77

## Criando as faixas etárias
bins= [ 20, 30, 40, 50, 60, 70, 80]
## Labels correspondentes a ordem da lista acima
labels = ['20 a 29', '30 a 39', '40 a 49', '50 a 59','60 a 69', '70 a 79']

## Criando coluna faixa etaria no dataframe e atribuindo os valores das faixas
df['fx_etaria'] = pd.cut( df['dob_age'], bins=bins, labels=labels, right=False)

## Criando um DF com o agrupamento de faixa etária e sexo para a criação do gráfico
df_fx_etaria = df.groupby(by=['fx_etaria'], dropna=False).count()[['gender']].rename(columns={'gender':'total'}).reset_index()

import plotly.express as px  # Importando biblioteca de gráficos

## Criando o objeto da figura do grafico
fig = px.histogram(df, x="fx_etaria")
## Mostrando o gráfico
fig.show()

21
78


### Etapa 5: Analisando dados com agrupamento 📊

- **Objetivo**: utilizar técnicas de agrupamento para descobrir usuários que moram no mesmo país e estado.
- **Descrição da solução**: uma função que recebe, como parâmetro, um `pandas.DataFrame` e retorna um `pandas.DataFrame` com as mesmas colunas, mas com os dados agrupados por país e estado.
- **Links úteis:**
  - Documentação do Pandas: https://pandas.pydata.org/docs/

In [10]:
## Função que recebe um DF e dois nomes de coluna como parametros e retorna um DF com o agrupamento de duas colunas caso essas existam no df passado como parametro
def group_country_state(df, col1, col2):
       
    lista_campos = [ col1, col2 ]
    lista_colunas = df.columns

    ## verificar se todos os campos da lista_campos estão contidos na lista_colunas
    check = all( item in lista_colunas for item in  lista_campos) ## Return true or false

    if check is True:
        df_group = df.groupby(by= [ col1, col2 ], dropna=False).count()[['gender']].rename( columns={'gender':'total'} )
        return df_group

table_group = group_country_state(df, 'location_country', 'location_state')
print(table_group )


                                               total
location_country location_state                     
Australia        Australian Capital Territory      9
                 New South Wales                   7
                 Northern Territory               10
                 Queensland                        9
                 South Australia                  12
...                                              ...
United States    Vermont                           2
                 Washington                        2
                 West Virginia                     2
                 Wisconsin                         1
                 Wyoming                           2

[517 rows x 1 columns]


### Etapa 6 (opcional): Particionando dados 🎼

- **Objetivo**: realizar o particionamento dos dados em formato Hive utilizando as informações de país e estado de cada usuário.
- **Descrição da solução**: uma função que recebe, como parâmetro, um `pandas.DataFrame` e salva todos os dados em arquivos CSV particionados por país e estado.
- **Links úteis:**
  - Documentação do Pandas: https://pandas.pydata.org/docs/
  - Documentação do Hive: https://hive.apache.org/
  - Documentação do BigQuery para dados particionados em Hive: https://cloud.google.com/bigquery/docs/hive-partitioned-queries-gcs
- **Exemplo para esclarecimento**: supondo que haja um `DataFrame` conforme o seguinte:

  ```python
      ano  mes sigla_uf dado
  0  2020    1       SP    a
  1  2021    2       SP    b
  2  2020    3       RJ    c
  3  2021    4       RJ    d
  4  2020    5       PR    e
  5  2021    6       PR    f
  6  2021    6       PR    g
  7  2025    9       PR    h
  ```

  Caso quiséssemos particionar o `DataFrame` utilizando as colunas `ano`, `mes` e `sigla_uf`, o resultado obtido seria a seguinte estrutura de diretórios:

  ![Exemplo de particionamento](./img/exemplo-particao.png)

  Cada arquivo gerado, então, teria o seguinte formato (esse em questão seria o `ano=2025/mes=9/sigla_uf=PR/data.csv`):

  ```python
    dado
  0    h
  ```

  Note que, no arquivo CSV gerado, as colunas referentes às informações utilizadas para particionamento são removidas.

In [11]:
## Função para criar as pastas por país e cidade
def create_partitions_country_state( df ):
    
    import os

    ## Pegando o caminho da pasta do script
    path_script = os.path.dirname(os.path.realpath("__file__"))

    # Criar se nao existir o diretorio
    if not os.path.exists(os.path.join( path_script,'partitions')):
        os.makedirs(os.path.join( path_script,'partitions')) 

    path_partitions = os.path.join( path_script,'partitions')

    ## Criado um df com agrupamento de cidade e pais com o index resetado para não conter index multiplo
    df2 = df.groupby(by= [ 'location_country','location_state' ], dropna=False).count()[['gender']].rename( columns={'gender':'total'} ).reset_index()

    ## Criar dicionário com cidade na chave e país no valor
    dicionario_cidade_capital = pd.Series(df2.location_country.values,index=df2.location_state).to_dict()

    ## iterar neste dicionario para criar os bancos a partir das chaves e valores do dicionario contendo o parâmetro de seleçao da consulta ao df principal
    for key, value in dicionario_cidade_capital.items():
        
        pais = value
        cidade = key
        filename = f"{pais}_{cidade}.csv" 

        path_pais = os.path.join( path_partitions,"country=" + pais )
        path_cidade = os.path.join( path_partitions, "country=" + pais, "state=" + cidade )

        
        if not os.path.exists( path_pais ):     ## Criar se nao existir o diretorio de país
            os.makedirs( path_pais )
        ## 
        if not os.path.exists( path_cidade ):   ## riar se nao existir o diretorio de cidades
            os.makedirs( path_cidade )

        ## Criar os CSV's dentro das pastas
        df.loc[ ( df.location_country == pais ) & ( df.location_state == cidade )].to_csv( path_cidade + "//" +filename )

try:
    ## Aplicando a função e criando as partições
    create_partitions_country_state( df )
except Exception as e:
    print(e)

### Etapa 7: Parametrizando seu código ⚙️

- **Objetivo:** nessa etapa, você deve parametrizar seu código para que ele seja executado com valores diversos fornecidos pelo usuário.
- **Descrição da solução:** a solução dessa etapa consiste em uma função principal que recebe diversos parâmetros e executa as diversas etapas descritas anteriormente em função dos parâmetros fornecidos. Note que essa etapa é crucial para que seu código se torne reutilizável.
- **Dicas:**
  - Tente pensar no maior número de parâmetros que sejam relevantes para sua pipeline de dados, sem afetar sua funcionalidade.
  - Colocar valores padrão para alguns desses parâmetros reduz o ônus do usuário de preenchê-los por conta própria.

In [20]:
## Importando as bibliotecas
import requests
import pandas as pd
import json
from pandas import json_normalize
import numpy as np
import seaborn as sns
import sys
import urllib.parse


## Para o exercício 07 vamos escolher alguns parâmetros da própria API de consumo e criar outros.
## gender, city, state, country, age_range

# def get_api_data( rows=1000, gender="", city="", state="", country=""):
def get_api_data( rows=1000, **kwargs ):
    """Função para baixar informações da API Random User (https://randomuser.me/)
 
    Args:
        rows (int): Número de registros baixados
            default: 1000
            options: Number between 1 and 5000
            example: 2000            


        gender (str): Gênero do usuário.
            default: ""
            options: female / male
            example: female           


        nat (str): Naturalidade do usuário. Pode adicionar mais de uma naturalidade separada por vírgula
            default: ""
            options: AU, BR, CA, CH, DE, DK, ES, FI, FR, GB, IE, IN, IR, MX, NL, NO, NZ, RS, TR, UA, US
            example: BR,US            
    
    Return: Dataframe 

    """

    ## Validacao de parametros
    try:
        if 1 <= rows <= 5000:
            results = f"results={rows}"
            # print( results ) 

            ## Setando o endpoint
            endpoint = f"https://randomuser.me/api/?results={rows}&"

            if kwargs:
                parametros = [ 'gender', 'nat' ]
                parametros_passados = list(kwargs.keys())

                print( f"parametros: {parametros}\nparametros passados:{parametros_passados}\n"  )

                ## Conferir se os parâmetros passados estão corretos.
                result =  all( elem in parametros  for elem in parametros_passados )

                ## Caso sejam, entre aqui
                if result:
                    print( "result\n" )

                    list_string_parametros = []
                    parameters_api = ""

                    if 'gender' in parametros_passados:
                        gender = f"gender='{kwargs['gender']}'"
                        list_string_parametros.append(gender)
                        print(f"gender está nos parametros! ( {gender} )")
                                
                    if 'nat' in parametros_passados:
                        nat = f"nat='{kwargs['nat']}'".strip().replace( " ","")

                        list_string_parametros.append(nat)
                        print(f"country está nos parametros! ( {nat} )")

                    parameters_api = "&".join( list_string_parametros )
                    ## remover as aspas simpoles
                    parameters_api = parameters_api.replace( "'","")
                    parameters_api = urllib.parse.quote( parameters_api ,safe="=,&")

                    print( parameters_api )     

                    
                    ## Montando a query URL
                    query = endpoint + parameters_api
                    print( query )
                   
                ## Senão informe o erro!
                else :
                    print("Parâmetros inválidos!!")

            else:
                ## Montando a query URL
                query = endpoint.rstrip('&') 
                print( query)

            r = requests.get( query )

            # Tranformar o objeto r em uma string json 
            dict= json.loads(r.text)

            # Normalizar a string json, pois existem objetos aninhados. Estes receberam o separador "_" 
            df = json_normalize( dict['results'], sep = "_" )

            return df

        else:
            print ("Quantidade de linhas maior ou menor que a permitida ( 1 a 5000" )
            sys.exit()            
                    
    except Exception as error:
        print ("Exception TYPE:", type(error))
        sys.exit()


## Extraindo as informações com os parâmetros
df_final = get_api_data( rows=1000, gender="female", nat="US,BR" )  

## Aplicando a função e criando as partições
create_partitions_country_state( df_final )

# print(df_final)

parametros: ['gender', 'nat']
parametros passados:['gender', 'nat']

result

gender está nos parametros! ( gender='female' )
country está nos parametros! ( nat='US,BR' )
gender=female&nat=US,BR
https://randomuser.me/api/?results=1000&gender=female&nat=US,BR


# CIE - Centro de Inteligência Epidemiológica
![](https://svs.rio.br/epirio/img/CIE_RODAPE.svg)